# Les parcours

On se propose ici de parcourir les graphes non-orientés d'abord en profondeur puis en largeur. Les algorithmes que nous écrirons devront permettre d’explorer un graphe au départ de n’importe quel sommet.

Lors du parcours, on peut évidemment (un chemin d'un sommet à un autre peut ne pas être unique) retomber sur un sommet déjà visité. Pour éviter une exploration qui ne terminerait jamais, on "marque" les sommets visités pour éviter de les ré-explorer.

Dans ce document, on ne se préoccupe pas de l'ordre dans lequel les voisins d'un sommet sont examinés.

Dans ce document, on choisit comme implémentation la dernière réalisée:

In [2]:
class Graphe:
    def __init__(self, sommets):
        self.sommets = sommets
        self._adjacence = {}
        
    def ajouter_arete(self, sommet_1, sommet_2):
        if sommet_1 in self._adjacence:
            self._adjacence[sommet_1].add(sommet_2)
        else:
            self._adjacence[sommet_1] = {sommet_2}
        
        if sommet_2 in self._adjacence:
            self._adjacence[sommet_2].add(sommet_1)
        else:
            self._adjacence[sommet_2] = {sommet_1}
    
    def voisins(self, sommet):
        if not sommet in self.sommets:
            return None
        else:
            return self._adjacence[sommet]

On utilise toujours le même graphe (celui de l'exercice d'aquariophilie)

![](graphviz3.png)

On dispose de ce graphe sous la forme d'un dictionnaire :

In [4]:
incompatibilites = {
    "A": ["B", "C", "D", "G", "H"],
    "B": ["A", "E", "F", "G"],
    "C": ["A", "D", "F", "G", "H"],
    "D": ["A", "C", "E", "H"],
    "E": ["B", "D", "F", "G"],
    "F": ["B", "C", "E"],
    "G": ["A", "B", "C", "E"],
    "H": ["A", "C", "D"]
}

Avant d'explorer par des parcours, on crée un objet `Graphe` qui rend compte de ce dictionnaire d'incompatibilités :

In [8]:
G = Graphe(incompatibilites.keys())

for poisson in incompatibilites:
    for voisin in incompatibilites[poisson]:
        G.ajouter_arete(poisson, voisin)

G.voisins('A')

{'B', 'C', 'D', 'G', 'H'}

## Parcours en profondeur

Principe de l'algorithme :
 - on crée une pile
 - on démarre d'un sommet qu'on empile
 - on le dépile et on le marque comme "visité"
 - on empile les sommets adjacents
 - tant que la pile n'est pas vide, on dépile (un sommet), on empile les sommets adjacents qui n'ont pas déjà été visités

In [32]:
class Pile:
    def __init__(self):
        self._pile = []
        
    def empiler(self, element):
        self._pile.append(element)
        
    def depiler(self):
        if self._pile:
            return self._pile.pop()
        
    def est_vide(self):
        return self._pile == []

In [36]:
def parcours_prof_v1(graphe, sommet_depart):
    sommets_visites = set()
    p = Pile()
    p.empiler(sommet_depart)
    while not p.est_vide():
        s = p.depiler()
        if s not in sommets_visites:
            print(s)
            sommets_visites.add(s)
            for v in graphe.voisins(s):
                if v not in sommets_visites:
                    p.empiler(v)
        
        
parcours_prof_v1(G, 'A')

A
D
C
H
G
B
F
E


In [38]:
def parcours_prof_v2(graphe, sommet_depart):
    sommets_visites = set()
    p = Pile()
    p.empiler(sommet_depart)
    while not p.est_vide():
        s = p.depiler()
        if s not in sommets_visites:
            print(s)
            sommets_visites.add(s)
            for v in graphe.voisins(s):
                p.empiler(v)
        s = p.depiler()
        
parcours_prof_v2(G, 'A')

A
B
F
H
C
G
E


In [39]:
def parcours_prof_v3(graphe, sommet_depart):
    sommets_visites = []
    p = Pile()
    s = sommet_depart
    while s is not None:
        print(s)
        sommets_visites.append(s)
        for v in graphe.voisins(s):
            if v not in sommets_visites:
                sommets_visites.append(v)
                p.empiler(v)
        s = p.depiler()
        
parcours_prof_v3(G, 'A')

A
D
E
F
B
H
C
G


## Parcours en largeur

Le principe de l'algorithme est identique mais la structure accueillant les voisins est une file.


In [15]:
class File:
    def __init__(self):
        self._pile = []
        
    def enfiler(self, element):
        self._pile.append(element)
        
    def defiler(self):
        if self._pile:
            return self._pile.pop(0)

In [16]:
def parcours_larg(graphe, sommet_depart):
    sommets_visites = set()
    p = File()
    s = sommet_depart
    while s is not None:
        if s not in sommets_visites:
            print(s)
            sommets_visites.add(s)
            for v in graphe.voisins(s):
                p.enfiler(v)
        s = p.defiler()
        
parcours_larg(G, 'A')

A
G
C
H
B
D
E
F


## Pour aller plus loin

### Tester vos codes avec l'arbre de descendance de Molly

In [43]:
descendance_molly = {
    "molly": ["ginny", "percy", "charlie", "fred1", "george", "ron", "bill"],
    "ginny": ["james", "albus", "lily"],
    "percy": ["lucy", "moly"],
    "george": ["roxanne", "fred2"],
    "ron": ["rose", "hugo"],
    "bill": ["victorie", "dominique", "louis"]
}

liste_sommets = []
for f in descendance_molly:
    if f not in liste_sommets:
        liste_sommets.append(f)
    for v in descendance_molly[f]:
        if v not in liste_sommets:
            liste_sommets.append(v)
        
G = Graphe(liste_sommets)

for f in descendance_molly:
    for v in descendance_molly[f]:
        G.ajouter_arete(f, v)
        
parcours_prof_v1(G, 'molly')

molly
bill
dominique
victorie
louis
charlie
ron
hugo
rose
percy
moly
lucy
fred1
ginny
james
lily
albus
george
fred2
roxanne


## Programmer la fonction de parcours en profondeur en récursif

In [49]:
def _explore(graphe, sommet, visites):
    print(sommet)
    visites.append(sommet)
    for s in graphe.voisins(sommet):
        if s not in visites:
            _explore(graphe, s, visites)
    
def parcours_prof(graphe):
    sommets_visites = []
    for s in graphe.sommets:
        if s not in sommets_visites:
            _explore(graphe, s, sommets_visites)

In [50]:
G = Graphe(incompatibilites.keys())

for poisson in incompatibilites:
    for voisin in incompatibilites[poisson]:
        G.ajouter_arete(poisson, voisin)

In [51]:
parcours_prof(G)

A
G
E
F
B
C
H
D
